In [1]:
from __future__ import print_function, division

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np

import pickle

class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 110

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])
        
        self.discriminator.summary()

        # Build the generator
        self.generator = self.build_generator()
        
        self.generator.summary()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])
        

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(384 * 4 * 4, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((4, 4, 384)))
        model.add(BatchNormalization())
        #model.add(UpSampling2D())
        model.add(Conv2DTranspose(192, kernel_size=5, strides=2, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        #model.add(UpSampling2D())
        model.add(Conv2DTranspose(96, kernel_size=5, strides=2, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        #model.add(UpSampling2D())
        model.add(Conv2DTranspose(self.channels, kernel_size=5, strides=2, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)
        
        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())
        model.add(Conv2D(32, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes, activation="softmax")(features)
                
        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):
        
        (X_train, y_train), (_, _) = cifar10.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        D_loss = []
        G_loss = []
        Acc = []

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # The labels of the digits that the generator tries to create an
            # image representation of
            sampled_labels = np.random.randint(0, 10, (batch_size, 1))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, sampled_labels])

            # Image labels. 0-9 
            img_labels = y_train[idx]

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, [valid, img_labels])
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))

            D_loss.append(d_loss[0])
            G_loss.append(g_loss[0])
            Acc.append(d_loss[3])
            
            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.save_model()
                self.sample_images(epoch)
        return D_loss, G_loss, Acc

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,:])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.discriminator, "discriminator")


if __name__ == '__main__':
    acgan = ACGAN()
    D_loss, G_loss, Acc = acgan.train(epochs=30000, batch_size=100, sample_interval=200)

W0911 11:21:44.119616 10848 deprecation.py:506] From c:\users\pedro_000\anaconda3\envs\deep_learning_1\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 16)        448       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 16, 16, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0

W0911 11:21:45.577870 10848 deprecation.py:323] From c:\users\pedro_000\anaconda3\envs\deep_learning_1\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 8192)         1574752     input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 1)            8193        sequential[1][0]                 
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10)           81930       sequential[1][0]                 
Total params: 1,664,875
Trainable params: 1,663,883
Non-trainable params: 992
________________

W0911 11:21:45.951230 10848 deprecation.py:506] From c:\users\pedro_000\anaconda3\envs\deep_learning_1\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

29512 [D loss: 1.058220, acc.: 55.50%, op_acc: 87.00%] [G loss: 0.897660]
29513 [D loss: 1.203562, acc.: 61.50%, op_acc: 80.00%] [G loss: 0.798540]
29514 [D loss: 1.200571, acc.: 64.50%, op_acc: 82.00%] [G loss: 0.878795]
29515 [D loss: 1.075704, acc.: 58.50%, op_acc: 86.50%] [G loss: 0.831153]
29516 [D loss: 1.260442, acc.: 62.00%, op_acc: 82.00%] [G loss: 0.821673]
29517 [D loss: 1.124006, acc.: 52.50%, op_acc: 86.00%] [G loss: 0.841872]
29518 [D loss: 1.194915, acc.: 57.50%, op_acc: 82.50%] [G loss: 0.837659]
29519 [D loss: 1.145520, acc.: 55.00%, op_acc: 82.00%] [G loss: 0.831344]
29520 [D loss: 1.163530, acc.: 58.00%, op_acc: 83.00%] [G loss: 0.802694]
29521 [D loss: 1.154812, acc.: 55.00%, op_acc: 86.00%] [G loss: 0.781735]
29522 [D loss: 1.346557, acc.: 52.00%, op_acc: 83.00%] [G loss: 0.936875]
29523 [D loss: 1.146689, acc.: 57.00%, op_acc: 83.50%] [G loss: 0.795579]
29524 [D loss: 1.237101, acc.: 59.50%, op_acc: 83.00%] [G loss: 0.910206]
29525 [D loss: 1.322713, acc.: 56.00%,

29623 [D loss: 1.247704, acc.: 64.50%, op_acc: 83.50%] [G loss: 0.930126]
29624 [D loss: 1.154341, acc.: 58.50%, op_acc: 83.00%] [G loss: 0.907768]
29625 [D loss: 1.239454, acc.: 62.50%, op_acc: 84.00%] [G loss: 0.855975]
29626 [D loss: 1.158635, acc.: 67.00%, op_acc: 82.00%] [G loss: 0.919882]
29627 [D loss: 1.164236, acc.: 54.00%, op_acc: 87.50%] [G loss: 0.902028]
29628 [D loss: 1.181834, acc.: 60.00%, op_acc: 81.50%] [G loss: 0.889420]
29629 [D loss: 1.244789, acc.: 58.50%, op_acc: 79.50%] [G loss: 0.873109]
29630 [D loss: 1.291264, acc.: 58.00%, op_acc: 80.50%] [G loss: 0.869008]
29631 [D loss: 1.151561, acc.: 62.00%, op_acc: 81.00%] [G loss: 0.864281]
29632 [D loss: 1.141066, acc.: 55.50%, op_acc: 85.50%] [G loss: 0.908045]
29633 [D loss: 1.207269, acc.: 60.50%, op_acc: 81.50%] [G loss: 0.793784]
29634 [D loss: 1.176221, acc.: 58.00%, op_acc: 78.00%] [G loss: 0.861184]
29635 [D loss: 1.223062, acc.: 72.00%, op_acc: 80.50%] [G loss: 0.924866]
29636 [D loss: 1.306404, acc.: 59.00%,

29734 [D loss: 1.120924, acc.: 55.50%, op_acc: 85.00%] [G loss: 0.858453]
29735 [D loss: 1.201738, acc.: 51.50%, op_acc: 82.50%] [G loss: 0.867545]
29736 [D loss: 1.154794, acc.: 64.50%, op_acc: 83.50%] [G loss: 0.889677]
29737 [D loss: 1.218195, acc.: 63.00%, op_acc: 83.00%] [G loss: 0.892727]
29738 [D loss: 1.249700, acc.: 56.50%, op_acc: 83.50%] [G loss: 0.861501]
29739 [D loss: 1.187978, acc.: 61.00%, op_acc: 81.50%] [G loss: 0.867675]
29740 [D loss: 1.183836, acc.: 61.50%, op_acc: 82.50%] [G loss: 0.877812]
29741 [D loss: 1.261249, acc.: 66.00%, op_acc: 82.50%] [G loss: 0.896878]
29742 [D loss: 1.233012, acc.: 57.50%, op_acc: 80.50%] [G loss: 0.849005]
29743 [D loss: 1.154495, acc.: 59.00%, op_acc: 83.50%] [G loss: 0.872022]
29744 [D loss: 1.244164, acc.: 61.00%, op_acc: 83.00%] [G loss: 0.864194]
29745 [D loss: 1.114953, acc.: 61.00%, op_acc: 85.50%] [G loss: 0.879255]
29746 [D loss: 1.167108, acc.: 57.50%, op_acc: 83.00%] [G loss: 0.917600]
29747 [D loss: 1.335523, acc.: 57.50%,

29845 [D loss: 1.231038, acc.: 53.00%, op_acc: 85.00%] [G loss: 0.931717]
29846 [D loss: 1.215972, acc.: 65.50%, op_acc: 81.50%] [G loss: 0.881952]
29847 [D loss: 1.232536, acc.: 60.50%, op_acc: 82.50%] [G loss: 0.915482]
29848 [D loss: 1.115551, acc.: 60.00%, op_acc: 82.50%] [G loss: 0.896876]
29849 [D loss: 1.183462, acc.: 59.50%, op_acc: 83.00%] [G loss: 0.905363]
29850 [D loss: 1.214292, acc.: 63.00%, op_acc: 80.50%] [G loss: 0.952087]
29851 [D loss: 1.171512, acc.: 60.50%, op_acc: 84.00%] [G loss: 0.968291]
29852 [D loss: 1.194571, acc.: 63.00%, op_acc: 82.00%] [G loss: 0.922829]
29853 [D loss: 1.316813, acc.: 56.00%, op_acc: 81.50%] [G loss: 0.942499]
29854 [D loss: 1.029689, acc.: 69.00%, op_acc: 85.00%] [G loss: 0.858223]
29855 [D loss: 1.176011, acc.: 61.50%, op_acc: 80.50%] [G loss: 0.946047]
29856 [D loss: 1.173120, acc.: 63.00%, op_acc: 82.50%] [G loss: 0.932317]
29857 [D loss: 1.265851, acc.: 58.00%, op_acc: 79.50%] [G loss: 0.920543]
29858 [D loss: 1.128461, acc.: 59.00%,

29956 [D loss: 1.117198, acc.: 55.00%, op_acc: 86.50%] [G loss: 0.911411]
29957 [D loss: 1.303841, acc.: 57.50%, op_acc: 78.50%] [G loss: 0.883113]
29958 [D loss: 1.193760, acc.: 62.50%, op_acc: 81.50%] [G loss: 0.899316]
29959 [D loss: 1.191945, acc.: 68.50%, op_acc: 80.50%] [G loss: 0.890252]
29960 [D loss: 1.247338, acc.: 63.50%, op_acc: 82.50%] [G loss: 0.878568]
29961 [D loss: 1.260798, acc.: 64.50%, op_acc: 81.50%] [G loss: 0.906134]
29962 [D loss: 1.220874, acc.: 59.00%, op_acc: 84.50%] [G loss: 0.889690]
29963 [D loss: 1.277687, acc.: 53.50%, op_acc: 80.00%] [G loss: 0.838762]
29964 [D loss: 1.121020, acc.: 53.00%, op_acc: 85.50%] [G loss: 0.933212]
29965 [D loss: 1.098255, acc.: 55.50%, op_acc: 85.00%] [G loss: 0.902359]
29966 [D loss: 1.345438, acc.: 53.00%, op_acc: 78.00%] [G loss: 0.897808]
29967 [D loss: 1.110184, acc.: 61.50%, op_acc: 84.00%] [G loss: 0.918766]
29968 [D loss: 1.145608, acc.: 50.50%, op_acc: 84.00%] [G loss: 0.866306]
29969 [D loss: 1.175797, acc.: 57.00%,